In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral-qna/quora_dataset.csv
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-v0.1-hf/1/generation_config.json


In [2]:
#complains a bunch about updated packages - seems to work fine
%pip install -q -U bitsandbytes
%pip install -q -U trl 
%pip install -q -U accelerate
%pip install -q -U transformers
%pip install -q -U peft
%pip install -q datasets==2.16.0

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires ptxcompiler, which is not installed.
cuml 24.6.1 requires c

In [3]:
import os
import torch, wandb
import numpy as np
import pandas as pd
import warnings
from datasets import Dataset
from IPython.display import Markdown, display
from IPython.utils import io

from peft import PeftModel, LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import BitsAndBytesConfig, AutoTokenizer, TrainingArguments, AutoModelForCausalLM, HfArgumentParser, TrainingArguments, pipeline, logging
from trl import SFTTrainer


2024-07-29 19:18:41.350426: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 19:18:41.350585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 19:18:41.499448: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_path = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


(True, True)

In [5]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [6]:
def display_formatted(input_text):
    input_text = input_text.replace("<s>", "").replace("</s>", "")
    user_start = input_text.find("[INST]") + len("[INST]")
    user_end = input_text.find("[/INST]")
    user_text = input_text[user_start:user_end].strip()
    llm_response = input_text[user_end + len("[/INST]"):].strip()
    
    formatted_text = f"<b>User:</b><br>{user_text}\n\n<b>LLM Response:</b><br>{llm_response}"

    display(Markdown(formatted_text))


def get_mistral_response(prompt):
    # Construct the prompt
    messages = [
        {"role": "user", "content": prompt},
    ]
    
    # Apply the chat template and tokenize
    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = model_inputs.to("cuda")
    
    # Generate the response with stopping criteria
    generated_ids = model.generate(
        model_inputs, 
        max_new_tokens=350, 
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,  # Ensure end-of-sequence token is used
        early_stopping=True
    )

    # Decode the generated ids
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return decoded[0]

In [7]:
from sklearn.model_selection import train_test_split
# Load the train and test data
df = pd.read_csv('/kaggle/input/mistral-qna/quora_dataset.csv')
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
test_df=test_df[:250]
train_df=train_df[:4000]

train_df['question'] = train_df['question'].astype(str)
train_df['answer'] = train_df['answer'].astype(str)
test_df['question'] = test_df['question'].astype(str)
test_df['answer'] = test_df['answer'].astype(str)

In [8]:
from datasets import Dataset

def prepare_dataset(df):
    chats = []
    for index, row in df.iterrows():
        chat = [
            {"role": "user", "content": row['question']},
            {"role": "assistant", "content": row['answer']},
        ]
        chats.append(chat)
    
    chat_dataset = Dataset.from_dict({"chat": chats})
    
    formatted_chat_data = chat_dataset.map(
        lambda x: {"formatted_chat": tokenizer.apply_chat_template(x["chat"], tokenize=False, add_generation_prompt=False)}
    )
    
    return formatted_chat_data

train_dataset = prepare_dataset(train_df)
test_dataset = prepare_dataset(test_df)


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [9]:
pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=983f9f7102b9b0968f100345e03c68458dbd10a0115fca4c7d914d436e150fec
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [10]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [11]:
import wandb
wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
wandb.init(project="mistral_fine_tune",
    job_type="training"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rajeevkhanna31167 (personal_007). Use `wandb login --relogin` to force relogin


In [12]:
from datasets import load_metric

rouge = load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions.argmax(-1)

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    rouge_output = rouge.compute(predictions=pred_str, references=labels_str)
    
    return {
        "rouge1": rouge_output["rouge1"]*100,
        "rouge2": rouge_output["rouge2"]*100,
        "rougeL": rouge_output["rougeL"]*100,
    }


/tmp/ipykernel_34/3440905356.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [13]:
# Training arguments with reduced batch size and gradient checkpointing
training_arguments = TrainingArguments(
    output_dir='./mistral_fine_tune',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=200,  # Adjust the steps for evaluation as needed
    logging_steps=100,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    group_by_length=True,
    report_to="wandb",
    dataloader_pin_memory=True,  # Optimize data loading
    dataloader_num_workers=4,  # Optimize data loading
    gradient_checkpointing=True,  # Enable gradient checkpointing

)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Function to perform manual garbage collection
def perform_gc():
    gc.collect()
    torch.cuda.empty_cache()

In [15]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    max_seq_length=256,
    dataset_text_field="formatted_chat",
    args=training_arguments,
    packing=False,
#     compute_metrics=compute_metrics
)
# results_before_training = trainer.evaluate()


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [16]:
# torch.cuda.empty_cache()
# perform_gc()

In [17]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before

Step,Training Loss,Validation Loss
200,1.357300,1.355378
400,1.207300,1.312295
600,1.267800,1.273623
800,1.229100,1.239618
1000,1.221100,1.218344
1200,1.241600,1.203349
1400,1.127700,1.189386
1600,1.103900,1.173675
1800,1.150100,1.165976
2000,1.141600,1.162935


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=2000, training_loss=1.2275247268676759, metrics={'train_runtime': 13052.5893, 'train_samples_per_second': 0.306, 'train_steps_per_second': 0.153, 'total_flos': 3.611466506067149e+16, 'train_loss': 1.2275247268676759, 'epoch': 1.0})

In [ ]:
# # Evaluate the model after training
# perform_gc()
# results_after_training = trainer.evaluate()
# print("Results after training:", results_after_training)

In [22]:
 trainer.model.push_to_hub('mistral_fine_tune', use_temp_dir=False)

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Feluda/mistral_fine_tune/commit/130efb92dd84f7a3d0cc38768e6269e9c5e7537b', commit_message='Upload model', commit_description='', oid='130efb92dd84f7a3d0cc38768e6269e9c5e7537b', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
 trainer.tokenizer.push_to_hub('mistral_fine_tune', use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Feluda/mistral_fine_tune/commit/3740d70015233e7ec33523b28d793e51a3502edd', commit_message='Upload tokenizer', commit_description='', oid='3740d70015233e7ec33523b28d793e51a3502edd', pr_url=None, pr_revision=None, pr_num=None)

In [24]:
new_model_name = "mistral-7b_fine_tuned"
trainer.model.save_pretrained(new_model_name)
trainer.tokenizer.save_pretrained(new_model_name)
wandb.finish()
model.config.use_cache = True

eval/loss,█▆▅▄▃▂▂▁▁▁
eval/runtime,▄▂█▇▃▃▁▃▂▂
eval/samples_per_second,██▁▁██████
eval/steps_per_second,██▁▁██████
train/epoch,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇████
train/grad_norm,▂▄█▄▃▄▂▄▄▄▄▄▆▃▇▃▃▆▃▁
train/learning_rate,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
train/loss,█▅▄▃▄▃▃▃▃▃▃▃▁▂▂▂▂▂▂▂
eval/loss,1.16294
eval/runtime,229.5767
